To use this Notebook, follow the instructions in [README.md](../../git/README.md)

In [1]:
import pandas as pd               # a data.frame handler like R
import folium                     # displaying maps
import os
import matplotlib.pyplot as plt

import osmnx as ox                # connection to OpenStreetMap
from selenium import webdriver    # for rendering in browser to save as .png
import time
import math
import PIL as pil                 # for saving gifs

In [2]:
# read data
current_file = globals()['_dh'][0]  # absolut path of this file working in jupyter notebook
df = pd.read_table(current_file + os.sep + 'data' + os.sep + 'trafficCounts.csv', sep =';', encoding='latin1')  # os.sep as os inedpendent sep
parcl = pd.read_table(current_file + os.sep + 'data' + os.sep + 'parcLot.csv', sep =';', encoding='latin1')  # os.sep as os inedpendent sep

In [18]:
df.head()
len(df.index)

1619664

In [3]:
tmp = df[df['DateTimeFrom'] == '2019-01-01T00:00:00+01:00'] # select data from that date
len(tmp) # number of rows
tmp['SiteCode'].unique() 
tmp = tmp[tmp['SiteCode'] < 1000]  # SiteCode are integers
agg = tmp.groupby('SiteCode').PW.agg('sum')   # aggregate PW
f = pd.merge(agg, tmp[['SiteCode','Geo Point']], how='inner',on='SiteCode')
f['SiteCode'].unique().tolist()  # list of unique entries in col SiteCode
# split Geo Point in lng an lat
f[['lat','lng']] = f['Geo Point'].str.split(',', expand=True) # split Geo Points into new cols
# for calculation circles
maxPW = f['PW'].max()
minPW = f['PW'].min()
meanPW = f['PW'].mean()


In [4]:
# code for parking lots
#parcl.head()
unique_parcl = pd.DataFrame(parcl['geo_point_2d'].unique())
unique_parcl[['lat','lng']] = unique_parcl[0].str.split(',', expand=True) # split Geo Points into new cols
unique_parcl.drop(0, axis=1, inplace=True)
unique_parcl

,lat,lng
0,47.5504299,7.5943046
1,47.5630411,7.5967098
2,47.5639644,7.5946604
3,47.5622725,7.5917937
4,47.561101,7.5824076
5,47.5592347,7.58658
6,47.5515968,7.593512
7,47.5458851,7.5884556
8,47.5651794,7.6089067
9,47.563241,7.602175


In [5]:
tmp.to_csv(os.path.join(current_file,'data','spots.csv'),sep=';', index=False)

In [6]:
# handisch points for first street
fromSBB = [[47.55129881490497, 7.587171969561436],[47.55245107418086, 7.586515336091145]]
toSBB = [[47.55129881490497, 7.587171969561436],[47.55050994586821, 7.587605347651829]]
heuwoog = [[47.55245107418086, 7.586515336091145],[47.55050994586821, 7.587605347651829]]

In [7]:
# function to calculate triangulars
def calcTriangle(ll, steps):
    # takes a list of 2 points with WGS koordinates [N,E], first is the starting point
    # steps is the number of points claculated in between, uneven number  
    # returns a list with tuples of list of points and a rotation
    resl = []
    
    # select points
    startN = ll[0][0]; startE = ll[0][1]; endN = ll[1][0]; endE = ll[1][1];
   
    # calulate step size
    stepN = (startN - endN) / steps
    stepE = (startE - endE) / steps

    # radius list
    rad = [0,8,10,10,8,0]
    
    oldN = startN
    oldE = startE
    
    # calculate new points
    for i in range(2,steps-1):
        # calculate new points
        newN = startN - i * stepN
        newE = startE - i * stepE
        
        # we count alpha (the angle) to the previous center of a triangle
        if i >= ((steps+1)/2) and oldN - newN > 0:
            rot = 90 + math.degrees(math.atan((oldE-newE)/(oldN-newN)))
        elif i >= ((steps+1)/2) and oldN - newN <= 0:
            rot = 270 + math.degrees(math.atan((oldE-newE)/(oldN-newN)))
        elif i < ((steps+1)/2) and oldN - newN > 0:
            rot = 270 + math.degrees(math.atan((oldE-newE)/(oldN-newN)))
        elif i < ((steps+1)/2) and oldN - newN <= 0:
            rot = 90 + math.degrees(math.atan((oldE-newE)/(oldN-newN)))
            
        resl.append(([newN, newE], rot, rad[i-1]))
        
        oldN = newN
        oldE = newE
           
    return resl

def plotDirection(l, steps, color, map_obj):
    ll= calcTriangle(l, steps)
    for p, rot, rad in ll:
        folium.RegularPolygonMarker(location=[p[0],p[1]], weight = 2, color=color, fill=True, fill_color=color, number_of_sides=3, radius=rad, rotation=rot).add_to(map_obj)


def plotDot(row, color, map_obj):
    #radius = row.PW / maxPW * 10
    folium.CircleMarker(location=[row.lat, row.lng],radius=6,color = color,fill=True, fillcolor = color, fill_opacity =1).add_to(map_obj)

    
def plotParking(row, color, map_obj):
    #radius = row.PW / maxPW * 10
    folium.Marker(location=[row.lat, row.lng],icon=folium.Icon(color=color, icon='car', prefix='fa')).add_to(map_obj)    

#def plotParking(row, color, map_obj):
#    #radius = row.PW / maxPW * 10
#    folium.Marker(location=[row.lat, row.lng],icon=folium.features.DivIcon(
#        icon_size = (50,50),
#        icon_anchor = (7,20),
#        html='<div style="font-size: 18pt; background-color:  blue; color : white">P</div>',)).add_to(map_obj)   

In [75]:
calcTriangle(toSBB,4)

47.5525495
0.0006049
-0.0003352


([[47.5525495, 7.5864495],
  [47.5519446, 7.5867847],
  [47.5513397, 7.587119899999999],
  [47.5507348, 7.5874551]],
 61)

In [78]:
calcTriangle(fromSBB,4)

47.550078
-0.0006249
0.0003424


([[47.550078, 7.5879091],
  [47.5507029, 7.5875667],
  [47.551327799999996, 7.5872243],
  [47.5519527, 7.5868819]],
 -61)

In [8]:
# the map
place_name = 'Basel, Switzerland'
graph = ox.graph_from_place(place_name)
nodes, streets = ox.graph_to_gdfs(graph)
#streets.head()
style = {'color': '#F7DC6F', 'weight':'1'}
m = folium.Map([47.54970167889083, 7.591000145233195], width=975, height =575,zoom_start=16, tiles='CartoDB Positron')
# plot points
unique_parcl.apply(plotParking, color='darkblue', map_obj = m, axis = 1)
#folium.PolyLine(toSBB, color='blue', weight=2, opacity=1).add_to(m)
#folium.PolyLine(fromSBB, color='red', weight=2, opacity=1).add_to(m)
#folium.RegularPolygonMarker(location=toSBB[1], weight = 2, color ='blue', fill=True, fill_color='blue', number_of_sides=3, radius=6, rotation=250).add_to(m)
plotDirection(heuwoog, 7, 'green', m)
#plotDirection(fromSBB, 4, ' red', m)

m


In [9]:
m1 = folium.Map([47.54970167889083, 7.591000145233195], width=975, height =575,zoom_start=14, tiles='CartoDB Positron')
#unique_parcl.apply(plotParking, color = 'blue', map_obj = m1, axis = 1)
unique_parcl.apply(plotDot, color = 'blue', map_obj = m1, axis = 1)
f.apply(plotDot, color = 'red', map_obj = m1, axis = 1)
m1

In [206]:
# Save as map as image
# download gecko driver for firefox from here - https://github.com/mozilla/geckodriver/releases
mapFname = 'out.html'
m.save(mapFname)
mapUrl = 'file://{0}/{1}'.format(os.getcwd(), mapFname)
# use selenium webdriver to save the html as png image
driver = webdriver.Firefox()
driver.set_window_size(1000, 700)
driver.set_window_position(0, 0)
driver.get(mapUrl)
# wait for 2 seconds for the maps and other assets to be loaded in the browser
time.sleep(2)
driver.save_screenshot(os.path.join('png','o.png'))
driver.quit()

In [207]:
from PIL import Image
images = []
# create list of all files in png folder
filelis t= [file for file in os.listdir('png') if file.endswith('.png')]
#img = Image.open('out.png')
filelist

['o.png']